In [1]:
from utils import read_infections_and_deaths
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

from june.records import RecordReader

No --data argument given - defaulting to:
/cosma6/data/dp004/dc-cues1/JUNE-private/data
No --configs argument given - defaulting to:
/cosma6/data/dp004/dc-cues1/JUNE-private/june/configs
--------------------------------------------------------------------------
By default, for Open MPI 4.0 and later, infiniband ports on a device
are not used by default.  The intent is to use UCX for these devices.
You can override this policy by setting the btl_openib_allow_ib MCA parameter
to true.

  Local host:              login7b
  Local adapter:           mlx5_0
  Local port:              1

--------------------------------------------------------------------------
--------------------------------------------------------------------------

  Local host:   login7b
  Local device: mlx5_0
--------------------------------------------------------------------------


In [2]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

plt.style.use(['science'])
plt.style.reload_library()

In [ ]:
read = RecordReader(
    results_path='/cosma6/data/dp004/dc-quer1/vaccine_paper_runs/baselines/results/run_007/'
)
people_df = read.table_to_df('population')

In [ ]:
%%time

infections_df, deaths_df = read_infections_and_deaths(run=7)


In [ ]:
age_bins = [0,12,16,18,40,60,80,100]


In [ ]:
mean_age = infections_df.groupby(['timestamp'])['age'].mean().rolling(window='5D').median()

In [ ]:
n_infections = infections_df.groupby(['timestamp']).size().rolling(window='5D').mean()

In [ ]:
df_age = pd.concat([n_infections, mean_age], axis=1)

In [ ]:
df_age.index = pd.to_datetime(df_age.index)

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))

df_age.reset_index().plot.scatter(
    'timestamp', 0, c='age',cmap='cividis',ax=ax,
    s=5
)
ax.set_xlabel('')
ax.set_ylabel('\# Infections')

In [ ]:
# Percent of daily infections by age group

In [ ]:
n_infected_by_age = infections_df.groupby(['timestamp',pd.cut(infections_df['age'],
            bins=age_bins,right=False)]).size()

In [ ]:
percent_infections_in_age_group = n_infected_by_age/infections_df.groupby(['timestamp']).size()

In [ ]:
fig, ax = plt.subplots(nrows=2,figsize=(6,4),sharex=True,
                       constrained_layout=True,
                       gridspec_kw = {'wspace': 0.,'hspace':0}
                      )

df_age.reset_index().plot.scatter(
    'timestamp', 0, c='age',cmap='cividis',ax=ax[0],
    s=5
)
ax[0].set_xlabel('')
ax[0].set_ylabel('\# Infections')

(100.*percent_infections_in_age_group).unstack().plot.area(stacked=True, cmap='cividis',alpha=0.8,ax=ax[1])
ax[1].set_ylabel('\% Infections in age group')

ax[1].legend(bbox_to_anchor=(1,1))
ax[1].set_xlabel('')
#fig.subplots_adjust(hspace=0)

In [ ]:
fig, ax = plt.subplots(nrows=2,figsize=(8,4),sharex=True,
                       constrained_layout=True,
                       gridspec_kw = {'wspace': 0.,'hspace':0}
                      )

(n_infected_by_age).unstack().plot.area(stacked=True, cmap='cividis',alpha=0.8,ax=ax[0],legend=False)
ax[0].set_xlabel('')
ax[0].set_ylabel('\# Infections')

(100.*percent_infections_in_age_group).unstack().plot.area(stacked=True, cmap='cividis',alpha=0.8,ax=ax[1],
                                                          legend=False)
ax[1].set_ylabel('\% Infections in age group')

L = ax[0].legend(bbox_to_anchor=(1,1))
L.get_texts()[0].set_text('0 to 12 years old')
L.get_texts()[1].set_text('12 to 16 years old')
L.get_texts()[2].set_text('16 to 18 years old')
L.get_texts()[3].set_text('18 to 40 years old')
L.get_texts()[4].set_text('40 to 60 years old')
L.get_texts()[5].set_text('60 to 80 years old')
L.get_texts()[6].set_text('80 to 100 years old')

ax[1].set_xlabel('')
#fig.subplots_adjust(hspace=0)
plt.savefig('figures/age_hists.pdf')